In [1]:
!pip install torch torchvision
!pip install wavio
!pip install sounddevice

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive/IMT Atlantique/Projet 3A /master/kitchen20"
%cd /content/drive/My Drive/IMT Atlantique/Projet 3A /master/kitchen20

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 audioUtils.py		  envnet.pyc	 kitch_env		 transforms.py
 config.ini		  esc.py	 __pycache__		 utils.py
'Copie de kitchen20.py'   __init__.py	 training.py
 envnet.py		  kitchen20.py	 training_script.ipynb
/content/drive/My Drive/IMT Atlantique/Projet 3A /master/kitchen20


In [ ]:
from envnet import EnvNet
from kitchen20 import Kitchen20
from torch.utils.data import DataLoader
import torch.nn as nn
import utils as U
import torch


# Model
model = EnvNet(20, True)
model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)


# Dataset
batchSize = 32
inputLength = 48000
transforms = []
transforms += [U.random_scale(1.25)]  # Strong augment
transforms += [U.padding(inputLength // 2)]  # Padding
transforms += [U.random_crop(inputLength)]  # Random crop
transforms += [U.normalize(float(2 ** 16 / 2))]  # 16 bit signed
transforms += [U.random_flip()]  # Random +-

trainData = Kitchen20(root='../',
                      transforms=transforms,
                      folds=[1,2,3,4,5,6,7,8],
                      overwrite=False,
                      audio_rate=44100,
                      use_bc_learning=False)
trainIter = DataLoader(trainData, batch_size=batchSize,
                       shuffle=True, num_workers=2)


inputLength = 64000
transforms = []
transforms += [U.padding(inputLength // 2)]  # Padding
transforms += [U.random_crop(inputLength)]  # Random crop
transforms += [U.normalize(float(2 ** 16 / 2))]  # 16 bit signed
transforms += [U.random_flip()]  # Random +-

valData = Kitchen20(root='../',
                    transforms=transforms,
                    folds=[9,],
                    audio_rate=44100,
                    overwrite=False,
                    use_bc_learning=False)
valIter = DataLoader(valData, batch_size=batchSize,
                     shuffle=True, num_workers=2)


for epoch in range(600):
    tAcc = tLoss = 0
    vAcc = vLoss = 0
    for x, y in trainIter:  # Train epoch
        model.train()
        x = x[:, None, None, :]
        x = x.to('cuda:0')
        y = y.to('cuda:0')
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)
        y_pred = y_pred[:, :, 0, 0]

        # Compute and print loss
        loss = criterion(y_pred, y.long())
        acc = (y_pred.argmax(dim=1).long() == y.long()).sum()

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tLoss += loss.item()
        tAcc += acc.item()/len(trainData)

    for x, y in valIter:  # Test epoch
        model.eval()
        x = x[:, None, None, :]
        x = x.to('cuda:0')
        y = y.to('cuda:0')
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)
        y_pred = y_pred[:, :, 0, 0]
        loss = criterion(y_pred, y.long())
        acc = (y_pred.argmax(dim=1).long() == y.long()).sum()
        vLoss += loss.item()
        vAcc += acc.item()/len(valData)


    # loss = loss / len(dataset)
    # acc = acc / float(len(dataset))
    print('epoch {} -- train: {}/{} -- val:{}/{}'.format(
        epoch, tAcc, tLoss, vAcc, vLoss))

epoch 0 -- train: 0.12181303116147313/97.97092843055725 -- val:0.0/3.187875509262085
epoch 1 -- train: 0.15675165250236073/94.00973343849182 -- val:0.0/3.1697351932525635
epoch 2 -- train: 0.2181303116147309/91.95808124542236 -- val:0.0/3.1046066284179688
epoch 3 -- train: 0.2341831916902738/88.9431459903717 -- val:0.0/3.0460920333862305
epoch 4 -- train: 0.22096317280453262/88.35655999183655 -- val:0.0/3.2392897605895996
epoch 5 -- train: 0.22474032105760147/87.95760774612427 -- val:0.0/3.060206651687622
epoch 6 -- train: 0.25212464589235123/86.58423566818237 -- val:0.09090909090909091/3.020418643951416
epoch 7 -- train: 0.23984891406987724/85.94967555999756 -- val:0.0/3.058861494064331
epoch 8 -- train: 0.2587346553352219/85.23419046401978 -- val:0.09090909090909091/2.9845855236053467
epoch 9 -- train: 0.28045325779036834/83.57538199424744 -- val:0.09090909090909091/3.025662422180176
epoch 10 -- train: 0.2672332389046271/83.15415334701538 -- val:0.18181818181818182/3.059786081314087


In [ ]:
testData = Kitchen20(root='../',
                    transforms=transforms,
                    folds=[10,],
                    audio_rate=44100,
                    overwrite=False,
                    use_bc_learning=False)
testIter = DataLoader(testData, batch_size=1,
                     shuffle=True, num_workers=2)

In [ ]:
testAcc = 0
    
for x, y in testIter:  # Test epoch
        model.eval()
        x = x[:, None, None, :]
        x = x.to('cuda:0')
        y = y.to('cuda:0')
        # Forward pass: Compute predicted y by passing x to the model
        y_test = model(x)
        y_test = y_test[:, :, 0, 0]
        print(y_test)
        print(y_test.argmax(dim=1))
        #loss = criterion(y_pred, y.long())
        acc = (y_test.argmax(dim=1).long() == y.long()).sum()
        #vLoss += loss.item()
        testAcc += acc.item()/len(testData)

In [ ]:
testAcc

In [ ]:
len(testData)

In [ ]:
y

In [ ]:
y_pred.argmax(dim=1).long()

In [ ]:
import numpy as np

data = np.load('../audio/44100.npz', allow_pickle=True)
lst = data.files

for item in lst:
    print(item)
    print(data[item])

In [ ]:
len(trainData)

In [ ]:
acc.item()

In [ ]:
for i in range(10):
  print(len(data[data.files[i]].item()['sounds']))